# Web-scraping: сбор данных из баз данных и интернет-источников

## Управление браузером с Selenium: введение

*Алла Тамбовцева, НИУ ВШЭ*

### Установка и подготовка к работе

Библиотека Selenium – библиотека для управления браузером с помощью Python. Она позволяет запускать браузер и имитировать действия пользователя в нём. 

Зачем это может понадобиться? Во-первых, такое «встраивание» в браузер в большинстве случаев позволяет решить проблему с возникающими капчами и иными ограничениями, так как настройки сайта не распознают, что запрос к исходному коду страницы производится автоматически. Во-вторых, необходимость имитации действий в браузере неизбежно возникает при обработке динамических веб-страниц, где некоторые элементы (окна, графики, таблицы) появляются только при определенных действиях пользователя, например, при скроллинге или наведении мышкой. В-третьих, библиотека может быть полезна в случаях, если доступ к API сайта или базы данных получить довольно сложно, но при работе в браузере информация доступна (в таком случае можем залогиниться через Python как пользователь и потихоньку выгружать данные).

Установим библиотеку:

In [ ]:
!pip install selenium

Сначала импортируем библиотеку полностью, проверим, что всё идёт по плану:

In [ ]:
import selenium

Теперь импортируем отдельные компоненты:
    
* модуль `webdriver`, нужен непосредственно для запуска браузера через Python;
* коллекция атрибутов для поиска элементов на странице `By`.

In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By

Запустим браузер Chrome средствами Selenium (в модуле есть функции для разных браузеров, но лучше всего работать с Chrome или Firefox):

In [ ]:
br = wd.Chrome()

Если вы исполняете код выше первый раз, это может занять некоторое время, так как функция `Chrome()` сначала скачает и установит подходящую версию драйвера для Chrome, который будет обеспечивать связь между Selenium и той версией браузера Chrome, которая установлена у вас на компьютере. В последующие разы всё должно происходить быстрее.

В результате запуска строки выше должно открыться новое окно браузера Chrome. Оно пустое и, скорее всего, с всплывающим предупреждением о том, что браузером управляет автоматизированное тестовое ПО. Это нормально, мы «имитировали» открытие браузера от лица пользователя, теперь, если мы будем в этом окне переходить по ссылкам на сайты, сайты не будут воспринимать запросы как автоматические (в отличие от запросов через модуль `requests`, из-за автоматических или слишком быстрых запросов, собственно, могут возникать те же капчи).

Открывшееся окно браузера в течение работы закрывать нельзя. Python будет там выполнять действия, а мы будем их отслеживать со стороны.
Приступим к работе.

### Базовый пример работы с Selenium

Начнём с простого примера – сайта книжного магазина «Библио-Глобус». Вообще этот сайт можно парсить и без Selenium, он пока не блокирует автоматические запросы, но зато другие сайты магазинов или онлайн-кинотеатров с похожей структурой умеют это делать. 

Отправим запрос – перейдём в открытом окне по ссылке на [главную страницу](https://www.biblio-globus.ru/) сайта:

In [ ]:
br.get("https://www.biblio-globus.ru/")

В окне должна открыться главная страница сайта. Найдём на странице поле для поиска интересующего товара. Для этого нам понадобится метод `.find_element()`, он применяется к объекту *WebDriver* (у нас `br`). Вообще в Selenium есть два метода для поиска, по аналогии с методами `.find()` и `.find_all()` в BeautifulSoup:

* `.find_element()` – поиск одного элемента, возвращает один результат, если подходящих элементов несколько, возвращается первый;
* `.find_elements()` – поиск нескольких элементов, возвращает список результатов.

Искать будем по id объекта, его можем найти в исходном коде страницы, в инструментах разработчика, или через расширение *SelectorGadget* (установить можно [здесь](https://chromewebstore.google.com/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb?pli=1)):

In [ ]:
search = br.find_element(By.ID, "SearchBooks")

В SelectorGadget id объекта отображается с `#`, классы с `.`, в нём же можно запросить более сложный запрос `XPATH`, но об этом поговорим позже. Сейчас в `search` у нас хранится объект Selenium, к которому можно применять различные методы. Так как мы нашли поле для поиска, которое можно заполнять, мы воспользуемся методом `.send_keys()`, который введёт в это поле текст. Запросим книги по Python:

In [ ]:
search.send_keys("Python")

В открытом окне должен появиться текст и выпадающее меню с названиями книг. Магия! 

**Примечание.** Если мы нашли объект некорректно, например, не само поле (в HTML обычно с тэгом `<input>`), а рамочку вокруг него или раздел, внутри которого это поле находится, то есть те элементы, которые не подразумевают интерактива в виде ввода значений, в ответ на `.send_keys()` мы получим ошибку `element not interactable`.

Осталось выполнить поиск. Найдём кнопку с лупой для активации поиска и кликнем на неё!

In [ ]:
# по аналогии ищется тоже по id

button = br.find_element(By.ID, "SearchButton")

Для кликания используется метод с соответствующим названием:

In [ ]:
button.click()

Готово! В окне браузера должен появиться список результатов. В следующем практикуме мы продолжим работу с ним.